<a href="https://colab.research.google.com/github/FNSY96/TerrorismEventsPrediction/blob/master/Graduation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python Imports**
PLEASE PUT ALL LIBRARY IMPORTS IN THIS CELL **ONLY**

In [0]:
!pip3 install pandas
!pip install -U -q PyDrive

In [0]:
import csv
import glob
import os
import io
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn import metrics
from sklearn import preprocessing
from sklearn import datasets
from sklearn import tree
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Data Set Imports

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

drive.mount("/content/drive", force_remount = True)
datadir  = 'drive/My Drive/GraduationProject/Data'

## **GTD Data Frame**

In [0]:
gtd_df = pd.read_csv(os.path.join(datadir, 'gtd.csv'), encoding = 'latin-1')
print(gtd_df.columns.values)

In [0]:
gtd_filtered_df = gtd_df[['country', 'country_txt','region','region_txt','city','provstate', 'targtype1','targtype1_txt', 'iyear','imonth', 'gname', 'weaptype1', 'weaptype1_txt','attacktype1_txt','attacktype1']]
# gtd_filtered_df = gtd_filtered_df.rename(index=str, columns={"city": "city_txt", "provstate": "provstate_txt", "gname": "gname_txt"})
# print(gtd_filtered_df)
# print(gtd_filtered_df.isna().sum())

## **GECON Data Frame**

In [0]:
# gecon_df = pd.read_csv(os.path.join(datadir, 'Gecon40_post_final_kh.csv'),encoding='latin-1')
# print(gecon_df.columns.values)

In [0]:
# gecon_filtered_df = gecon_df[['AREA', 'COUNTRY', 'DIS_LAKE','DIS_MAJOR_RIVER']]

# print(gecon_filtered_df.isna().sum())

# print(gecon_filtered_df)

## **GeoEPR Data Frame**

In [0]:
geo_df = pd.read_csv(os.path.join(datadir, 'GeoEPR-2018.1.csv'),encoding='latin-1')
# print(geo_df.columns.values)
# # statename, group
# print(geo_df)

In [0]:
geo_filtered_df = geo_df[['statename','group']]
geo_filtered_df = geo_filtered_df.rename(index=str, columns={"statename": "country_txt"})
# print(geo_filtered_df)
# print(geo_filtered_df.isna().sum())
# print(geo_filtered_df.groupby('country_txt').count())

In [10]:
ethnic_count = geo_filtered_df.groupby('country_txt').count()
countries = pd.DataFrame(geo_filtered_df['country_txt'].drop_duplicates().sort_values())
ethnic_count = pd.concat([ethnic_count.reset_index(drop=True),countries.reset_index(drop=True)], axis=1)
ethnic_count = ethnic_count.rename(index=str, columns={"group":"group_count"})
print(ethnic_count.columns.values)

['group_count' 'country_txt']


## **Happiness World Report Data Frame**

In [0]:
happy_df_17 = pd.read_csv(os.path.join(datadir, 'Happiness2017.csv'),encoding='latin-1')
happy_df_16 = pd.read_csv(os.path.join(datadir, 'Happiness2016.csv'),encoding='latin-1')
happy_df_15 = pd.read_csv(os.path.join(datadir, 'Happiness2015.csv'),encoding='latin-1')

happy_df_17 = happy_df_17.rename(index=str, columns={"Country": "country_txt", "Happiness.Rank": "Happiness.Rank17", "Happiness.Score": "Happiness.Score17", "Freedom": "Freedom17", "Economy..GDP.per.Capita.": "Economy17"})
happy_df_16 = happy_df_16.rename(index=str, columns={"Country": "country_txt", "Happiness Rank": "Happiness.Rank16", "Happiness Score": "Happiness.Score16", "Freedom": "Freedom16", "Economy (GDP per Capita)": "Economy16"})
happy_df_15 = happy_df_15.rename(index=str, columns={"Country": "country_txt", "Happiness Rank": "Happiness.Rank15", "Happiness Score": "Happiness.Score15", "Freedom": "Freedom15", "Economy (GDP per Capita)": "Economy15"})

In [0]:
happiness_filtered = [happy_df_15[['country_txt','Happiness.Score15', 'Freedom15', 'Economy15']],
                      happy_df_16[['country_txt','Happiness.Score16', 'Freedom16', 'Economy16']], 
                      happy_df_17[['country_txt','Happiness.Score17', 'Freedom17', 'Economy17']]]

# Merging Data

In [13]:
# merging gtd with happiness
gtd_merged_happiness = gtd_filtered_df
for i in range(len(happiness_filtered)):
  gtd_merged_happiness = pd.merge(gtd_merged_happiness, happiness_filtered[i],on='country_txt')
print(gtd_merged_happiness.shape)

# merging gtd & happiness with ethnic
gtd_merged_happiness_ethnic = pd.merge(gtd_merged_happiness, ethnic_count, on='country_txt')
print(gtd_merged_happiness_ethnic.shape)

(171630, 24)
(167161, 25)


In [14]:
# selecting Middle East & North Africa region
# MENA stands for Middle East and North Africa region
mena = gtd_merged_happiness_ethnic.loc[gtd_merged_happiness_ethnic['region_txt'] == 'Middle East & North Africa']
print(mena.shape)

(45846, 25)


In [15]:
# Number of distinct terrorist groups in the MENA region = 636
# print(np.unique(mena[['gname']].values))
print(np.unique(mena[['gname']]).size)

636


#Creating MENA Dataframe

### removing nan

In [16]:
# drop rows with nan in mena
print(mena.shape)
mena_cleaned = mena.dropna()
print(mena_cleaned.shape)

(45846, 25)
(45760, 25)


###Thresholding

In [17]:
# select desired labels
mena_filtered = mena_cleaned[(mena_cleaned['gname'] == "Islamic State of Iraq and the Levant (ISIL)") | (mena_cleaned['gname'] == "Kurdistan Workers' Party (PKK)") | (mena_cleaned['gname'] == "Houthi extremists (Ansar Allah)") | (mena_cleaned['gname'] == "Al-Qaida in the Arabian Peninsula (AQAP)") | (mena_cleaned['gname'] == "Al-Qaida in Iraq")]

print(mena_filtered['gname'].value_counts())
print(mena_filtered['gname'].size)

Islamic State of Iraq and the Levant (ISIL)    5553
Kurdistan Workers' Party (PKK)                 2137
Houthi extremists (Ansar Allah)                1062
Al-Qaida in the Arabian Peninsula (AQAP)       1016
Al-Qaida in Iraq                                636
Name: gname, dtype: int64
10404


### Data Conversion

In [18]:
# convert categorical attributes to numerical

label_encoder = preprocessing.LabelEncoder()
string_attributes = ['city', 'provstate', 'gname']
mena_final = mena_filtered
mena_final = mena_final.drop(["gname", "provstate", "city"], axis=1)
print(mena_final.shape)
print(mena_final.columns)

for i in range(len(string_attributes)):
  attribute_name = string_attributes[i]
  attribute_list = mena_filtered[attribute_name]
  attribute_encoded = label_encoder.fit_transform(attribute_list)
  value_zipped_encoded_list = list(zip(attribute_list,attribute_encoded))
  value_zipped_encoded_df = pd.DataFrame(value_zipped_encoded_list)
  modified_attribute_name = attribute_name + '_numerical'
  value_zipped_encoded_df.columns = [attribute_name, modified_attribute_name]
  zipped_df = pd.DataFrame(value_zipped_encoded_df)
  print(zipped_df.shape)
  print(mena_final.shape)
  mena_final.reset_index(drop=True, inplace=True)
  zipped_df.reset_index(drop=True, inplace=True)
  mena_final = pd.concat([mena_final, zipped_df], axis = 1)
  print("after concat")
  print(mena_final.shape)
  print("*************")
  
mena_final = mena_final.dropna()
print(mena_final.shape)


(10404, 22)
Index(['country', 'country_txt', 'region', 'region_txt', 'targtype1',
       'targtype1_txt', 'iyear', 'imonth', 'weaptype1', 'weaptype1_txt',
       'attacktype1_txt', 'attacktype1', 'Happiness.Score15', 'Freedom15',
       'Economy15', 'Happiness.Score16', 'Freedom16', 'Economy16',
       'Happiness.Score17', 'Freedom17', 'Economy17', 'group_count'],
      dtype='object')
(10404, 2)
(10404, 22)
after concat
(10404, 24)
*************
(10404, 2)
(10404, 24)
after concat
(10404, 26)
*************
(10404, 2)
(10404, 26)
after concat
(10404, 28)
*************
(10404, 28)


### removing categorical columns

In [19]:
mena_final = mena_final.drop(["gname", "country_txt", "region_txt", "targtype1_txt", "provstate", "city", "weaptype1_txt", "attacktype1_txt", "imonth", "region"], axis=1)
print(mena_final.shape)

(10404, 18)


###removing records before year 2000

In [20]:
mena_final = mena_final[mena_final['iyear'] >= 2000]
print(mena_final.shape)

(9490, 18)


#Divide Data into Train & Test

In [0]:
mena_gtd = mena_final.drop(['Happiness.Score15', 'Freedom15', 'Economy15','Happiness.Score16', 'Freedom16', 'Economy16','Happiness.Score17', 'Freedom17', 'Economy17', 'group_count'], axis=1)
mena_gtd_happiness = mena_final.drop(['group_count'], axis=1)
mena_gtd_ethnic = mena_final.drop(['Happiness.Score15', 'Freedom15', 'Economy15','Happiness.Score16', 'Freedom16', 'Economy16','Happiness.Score17', 'Freedom17', 'Economy17'], axis=1)
mena_gtd_happiness_ethnic = mena_final
#print(mena_gtd_happiness_ethnic.shape)


In [0]:
# divide dataset into train and test
def train_test(mena_classification):
  
  mena_split_train = mena_classification[mena_classification['iyear'] < 2017]
  mena_split_test = mena_classification[mena_classification['iyear'] >= 2017]
  
  mena_split_train = mena_split_train.drop("iyear", axis = 1)
  mena_split_test = mena_split_test.drop("iyear", axis = 1)


  labels_train = mena_split_train['gname_numerical']
  labels_test = mena_split_test['gname_numerical']
  
  data_train = mena_split_train.loc[:, mena_split_train.columns != 'gname_numerical']
  data_test = mena_split_test.loc[:, mena_split_test.columns != 'gname_numerical']

  print(mena_split_train.shape)
  print(mena_split_test.shape)


  print(labels_train.shape)
  print(labels_train.shape)
  
  return labels_train, labels_test, data_train, data_test

In [23]:
labels_train1, labels_test1, data_train1, data_test1 = train_test(mena_gtd)
labels_train2, labels_test2, data_train2, data_test2 = train_test(mena_gtd_happiness)
labels_train3, labels_test3, data_train3, data_test3 = train_test(mena_gtd_ethnic)
labels_train4, labels_test4, data_train4, data_test4 = train_test(mena_gtd_happiness_ethnic)

print(mena_final)

(7842, 7)
(1648, 7)
(7842,)
(7842,)
(7842, 16)
(1648, 16)
(7842,)
(7842,)
(7842, 8)
(1648, 8)
(7842,)
(7842,)
(7842, 17)
(1648, 17)
(7842,)
(7842,)
       country  targtype1  iyear  weaptype1  attacktype1  Happiness.Score15  \
0          102          1   2005          6            3              5.192   
1          102          1   2005          6            3              5.192   
2          102          1   2005          6            3              5.192   
3          102          3   2015          5            2              5.192   
4          102          4   2016          6            3              5.192   
5          102         17   2016          6            3              5.192   
6          102          3   2016          5            2              5.192   
7          102          3   2016          5            2              5.192   
8          102         18   2016          6            5              5.192   
9           60          3   2014          5            2      

#Classifiers

###KNN

In [0]:
def knn_classifier(labels_train, labels_test, data_train, data_test):
  
  scaler = StandardScaler()  
  scaler.fit(data_train)

  data_train = scaler.transform(data_train)  
  data_test = scaler.transform(data_test)  

  acc = 0

  for i in range(3,1000):

    # Create KNN Classifier
    knn = KNeighborsClassifier(n_neighbors = i)

    # Train the model using the training sets
    knn.fit(data_train, labels_train)

    # Predict the response for test dataset
    y_pred = knn.predict(data_test)

    # Model Accuracy, how often is the classifier correct?
    curr_acc =  metrics.accuracy_score(labels_test, y_pred)
    if curr_acc > acc:
      acc = curr_acc
      print("Accuracy:", acc)
      print(confusion_matrix(labels_test, y_pred))  
      print(classification_report(labels_test, y_pred))
      print(i)


###Naïve Bayes

In [0]:
def naive_bayes_classifier(labels_train, labels_test, data_train, data_test):
  
  # Create a  Naive Bayes Classifier
  clf = GaussianNB()

  # Train the model using the training sets
  clf.fit(data_train, labels_train)

  # Predict the response for test dataset
  y_pred = clf.predict(data_test)

  print("Accuracy:", metrics.accuracy_score(labels_test, y_pred))
  print(confusion_matrix(labels_test, y_pred))  
  print(classification_report(labels_test, y_pred))

### Random Forests

In [0]:
def random_forest_classifier(labels_train, labels_test, data_train, data_test):
  
  acc = 0
  for i in range(3,1000):
    # Create a Gaussian Classifier
    clf = RandomForestClassifier(n_estimators = i)

    # Train the model using the training sets y_pred=clf.predict(X_test)
    clf.fit(data_train, labels_train)    
    y_pred=clf.predict(data_test)

    curr_acc =  metrics.accuracy_score(labels_test, y_pred)
    if curr_acc > acc:
      acc = curr_acc
      print("Accuracy:", acc)
      print(confusion_matrix(labels_test, y_pred))  
      print(classification_report(labels_test, y_pred))
      print(i)

###Decision tree


In [0]:
def decision_tree_classifier(labels_train, labels_test, data_train, data_test):
 
  clf = tree.DecisionTreeClassifier()
  clf.fit(data_train, labels_train)    #Train the model using the training sets y_pred=clf.predict(X_test)
  y_pred=clf.predict(data_test)

  print("Accuracy:", metrics.accuracy_score(labels_test, y_pred))
  print(confusion_matrix(labels_test, y_pred))  
  print(classification_report(labels_test, y_pred))

  # CART (Classification and Regression Trees) is very similar to C4.5, 
  # but it differs in that it supports numerical target variables (regression) and does not compute rule sets. 
  # CART constructs binary trees using the feature and threshold that yield the largest information gain at each node.
  # scikit-learn uses an optimised version of the CART algorithm.

###SVM

In [0]:
def svm_classifier(labels_train, labels_test, data_train, data_test):
  
  scaler = StandardScaler()  
  scaler.fit(data_train)

  data_train = scaler.transform(data_train)  
  data_test = scaler.transform(data_test)

  print(data_train.shape)
  print(labels_train.shape)
  print(data_test.shape)
  print(labels_test.shape)
  
  print(labels_train)
  print(data_train)

  
  #Create a svm Classifier
  clf = svm.SVC(kernel = 'linear') # Linear Kernel
  print("create")
  
  #Train the model using the training sets
  clf.fit(data_train, labels_train)
  print("fit")
  
  #Predict the response for test dataset
  y_pred = clf.predict(data_test)
  print("predict")

 # Model Accuracy: how often is the classifier correct?
  print("Accuracy:", metrics.accuracy_score(labels_test, y_pred))
  print(confusion_matrix(labels_test, y_pred))  
  print(classification_report(labels_test, y_pred))

# Calling Classifiers

### KNN

In [29]:
# training knn classifier on gtd only
knn_classifier(labels_train1, labels_test1, data_train1, data_test1)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  import sys
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.7512135922330098
[[  0   0   0   0   0]
 [  0  39   2   1   2]
 [  0  44 104   8   2]
 [274   2   5 973  37]
 [  1   9   4  19 122]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.41      0.89      0.57        44
           2       0.90      0.66      0.76       158
           3       0.97      0.75      0.85      1291
           4       0.75      0.79      0.77       155

   micro avg       0.75      0.75      0.75      1648
   macro avg       0.61      0.62      0.59      1648
weighted avg       0.93      0.75      0.83      1648

3
Accuracy: 0.8258495145631068
[[   0    0    0    0    0]
 [   0   38    3    0    3]
 [   2   32  114    6    4]
 [ 156    3    6 1089   37]
 [   1    6    9   19  120]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.48      0.86      0.62        44
           2       0.86      0.72      0.79       158
           3       0.98      0.84      0.91      1291
           4       0.73      0.77      0.75       155

   micro avg       0.83      0.83      0.83      1648
   macro avg       0.61      0.64      0.61      1648
weighted avg       0.93      0.83      0.87      1648

5
Accuracy: 0.8343446601941747
[[   0    0    0    0    0]
 [   0   35    6    1    2]
 [   0   36  103   13    6]
 [ 130    3    6 1110   42]
 [   1    3    8   16  127]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.45      0.80      0.58        44
           2       0.84      0.65      0.73       158
           3       0.97      0.86      0.91      1291
           4       0.72      0.82      0.77       155

   micro avg 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.8671116504854369
[[   0    0    0    0    0]
 [   0   24    4    8    8]
 [   0   17   84   19   38]
 [   1    4   26 1216   44]
 [   0    4   19   27  105]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.49      0.55      0.52        44
           2       0.63      0.53      0.58       158
           3       0.96      0.94      0.95      1291
           4       0.54      0.68      0.60       155

   micro avg       0.87      0.87      0.87      1648
   macro avg       0.52      0.54      0.53      1648
weighted avg       0.87      0.87      0.87      1648

212


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.8677184466019418
[[  25    4    7    8]
 [  18   84   17   39]
 [   4   27 1217   43]
 [   4   20   27  104]]
              precision    recall  f1-score   support

           1       0.49      0.57      0.53        44
           2       0.62      0.53      0.57       158
           3       0.96      0.94      0.95      1291
           4       0.54      0.67      0.60       155

   micro avg       0.87      0.87      0.87      1648
   macro avg       0.65      0.68      0.66      1648
weighted avg       0.88      0.87      0.87      1648

225
Accuracy: 0.8683252427184466
[[  26    4    7    7]
 [  18   84   17   39]
 [   4   27 1217   43]
 [   4   20   27  104]]
              precision    recall  f1-score   support

           1       0.50      0.59      0.54        44
           2       0.62      0.53      0.57       158
           3       0.96      0.94      0.95      1291
           4       0.54      0.67      0.60       155

   micro avg       0.87      0.87      0.87  

KeyboardInterrupt: ignored

In [0]:
# training knn classifier on gtd & happiness
knn_classifier(labels_train2, labels_test2, data_train2, data_test2)

In [0]:
# training knn classifier on gtd & ethnic
knn_classifier(labels_train3, labels_test3, data_train3, data_test3)

In [0]:
# training knn classifier on gtd, happiness & ethnic
knn_classifier(labels_train4, labels_test4, data_train4, data_test4)

### Naive Bayes

In [39]:
# training naive classifier on gtd only
naive_bayes_classifier(labels_train1, labels_test1, data_train1, data_test1)

Accuracy: 0.3446601941747573
[[  0   0   0   0   0]
 [  0  41   3   0   0]
 [  0  77  52  29   0]
 [817   0  28 347  99]
 [  7   0  16   4 128]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.35      0.93      0.51        44
           2       0.53      0.33      0.40       158
           3       0.91      0.27      0.42      1291
           4       0.56      0.83      0.67       155

   micro avg       0.34      0.34      0.34      1648
   macro avg       0.47      0.47      0.40      1648
weighted avg       0.83      0.34      0.44      1648



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
# training naive classifier on gtd & happiness
naive_bayes_classifier(labels_train2, labels_test2, data_train2, data_test2)

In [0]:
# training naive classifier on gtd & ethnic
naive_bayes_classifier(labels_train3, labels_test3, data_train3, data_test3)

In [0]:
# training naive classifier on gtd, happiness & ethnic
naive_bayes_classifier(labels_train4, labels_test4, data_train4, data_test4)

###Random Forest

In [0]:
# training random forest classifier on gtd only
random_forest_classifier(labels_train1, labels_test1, data_train1, data_test1)

In [0]:
# training random forest classifier on gtd & happiness
random_forest_classifier(labels_train2, labels_test2, data_train2, data_test2)

In [0]:
# training random forest classifier on gtd & ethnic
random_forest_classifier(labels_train3, labels_test3, data_train3, data_test3)

In [0]:
# training random forest classifier on gtd, happiness & ethnic
random_forest_classifier(labels_train4, labels_test4, data_train4, data_test4)

### Decision Tree

In [0]:
# training decision tree classifier on gtd only
decision_tree_classifier(labels_train1, labels_test1, data_train1, data_test1)

In [0]:
# training decision tree classifier on gtd & happiness
decision_tree_classifier(labels_train2, labels_test2, data_train2, data_test2)

In [0]:
# training decision tree classifier on gtd & ethnic
decision_tree_classifier(labels_train3, labels_test3, data_train3, data_test3)

In [0]:
# training decision tree classifier on gtd, happiness & ethnic
decision_tree_classifier(labels_train4, labels_test4, data_train4, data_test4)

###SVM

In [33]:
# training svm classifier on gtd only
svm_classifier(labels_train1, labels_test1, data_train1, data_test1)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  import sys


(7842, 6)
(7842,)
(1648, 6)
(1648,)
0        0
1        0
2        0
3        3
4        3
5        3
6        3
7        3
8        3
9        3
10       3
11       3
12       3
13       3
14       3
15       3
16       3
17       3
18       3
19       3
20       3
21       3
22       3
23       3
24       3
25       3
26       3
27       3
28       3
29       3
        ..
10201    2
10202    1
10203    2
10204    2
10205    2
10206    2
10207    2
10208    2
10209    2
10210    2
10211    2
10212    1
10213    2
10214    2
10215    2
10216    2
10217    2
10218    2
10219    2
10220    2
10221    2
10222    2
10223    2
10224    2
10225    2
10226    2
10227    1
10228    1
10229    2
10230    2
Name: gname_numerical, Length: 7842, dtype: int64
[[-0.77395716 -1.30076745 -0.39136671 -0.36389531 -1.40528775 -1.05382212]
 [-0.77395716 -1.30076745 -0.39136671 -0.36389531 -1.40528775 -1.05382212]
 [-0.77395716 -1.30076745 -0.39136671 -0.36389531 -1.40528775 -1.05382212]
 ...
 [ 1.30087293

In [34]:
# training svm classifier on gtd & happiness
svm_classifier(labels_train2, labels_test2, data_train2, data_test2)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


(7842, 15)
(7842,)
(1648, 15)
(1648,)
0        0
1        0
2        0
3        3
4        3
5        3
6        3
7        3
8        3
9        3
10       3
11       3
12       3
13       3
14       3
15       3
16       3
17       3
18       3
19       3
20       3
21       3
22       3
23       3
24       3
25       3
26       3
27       3
28       3
29       3
        ..
10201    2
10202    1
10203    2
10204    2
10205    2
10206    2
10207    2
10208    2
10209    2
10210    2
10211    2
10212    1
10213    2
10214    2
10215    2
10216    2
10217    2
10218    2
10219    2
10220    2
10221    2
10222    2
10223    2
10224    2
10225    2
10226    2
10227    1
10228    1
10229    2
10230    2
Name: gname_numerical, Length: 7842, dtype: int64
[[-7.73957156e-01 -1.30076745e+00 -3.91366712e-01 ... -6.71243985e-04
  -1.40528775e+00 -1.05382212e+00]
 [-7.73957156e-01 -1.30076745e+00 -3.91366712e-01 ... -6.71243985e-04
  -1.40528775e+00 -1.05382212e+00]
 [-7.73957156e-01 -1.30076745e+

In [31]:
# training svm classifier on gtd & ethnic
svm_classifier(labels_train3, labels_test3, data_train3, data_test3)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  import sys


(7842, 7)
(7842,)
(1648, 7)
(1648,)
0        0
1        0
2        0
3        3
4        3
5        3
6        3
7        3
8        3
9        3
10       3
11       3
12       3
13       3
14       3
15       3
16       3
17       3
18       3
19       3
20       3
21       3
22       3
23       3
24       3
25       3
26       3
27       3
28       3
29       3
        ..
10201    2
10202    1
10203    2
10204    2
10205    2
10206    2
10207    2
10208    2
10209    2
10210    2
10211    2
10212    1
10213    2
10214    2
10215    2
10216    2
10217    2
10218    2
10219    2
10220    2
10221    2
10222    2
10223    2
10224    2
10225    2
10226    2
10227    1
10228    1
10229    2
10230    2
Name: gname_numerical, Length: 7842, dtype: int64
[[-0.77395716 -1.30076745 -0.39136671 ... -0.94071701 -1.40528775
  -1.05382212]
 [-0.77395716 -1.30076745 -0.39136671 ... -0.94071701 -1.40528775
  -1.05382212]
 [-0.77395716 -1.30076745 -0.39136671 ... -0.94071701 -1.40528775
  -1.05382212]


In [32]:
# training svm classifier on gtd, happiness & ethnic
svm_classifier(labels_train4, labels_test4, data_train4, data_test4)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


(7842, 16)
(7842,)
(1648, 16)
(1648,)
0        0
1        0
2        0
3        3
4        3
5        3
6        3
7        3
8        3
9        3
10       3
11       3
12       3
13       3
14       3
15       3
16       3
17       3
18       3
19       3
20       3
21       3
22       3
23       3
24       3
25       3
26       3
27       3
28       3
29       3
        ..
10201    2
10202    1
10203    2
10204    2
10205    2
10206    2
10207    2
10208    2
10209    2
10210    2
10211    2
10212    1
10213    2
10214    2
10215    2
10216    2
10217    2
10218    2
10219    2
10220    2
10221    2
10222    2
10223    2
10224    2
10225    2
10226    2
10227    1
10228    1
10229    2
10230    2
Name: gname_numerical, Length: 7842, dtype: int64
[[-0.77395716 -1.30076745 -0.39136671 ... -0.94071701 -1.40528775
  -1.05382212]
 [-0.77395716 -1.30076745 -0.39136671 ... -0.94071701 -1.40528775
  -1.05382212]
 [-0.77395716 -1.30076745 -0.39136671 ... -0.94071701 -1.40528775
  -1.05382212